# Сравнение с SQL

Поскольку многие потенциальные пользователи pandas немного знакомы с SQL, эта страница предназначена для предоставления некоторых примеров того, как различные операции SQL будут выполняться с использованием pandas.

Если вы новичок в пандах, вы можете сначала прочитать 10 Minutes to pandas, чтобы ознакомиться с библиотекой.

Как обычно, импортируем pandas и NumPy следующим образом:

In [1]:
import pandas as pd

import numpy as np

В большинстве примеров будет использоваться набор данных `tips`, найденный в тестах pandas. Мы прочитаем данные в DataFrame под названием `tips` и предположим, что у нас есть таблица базы данных с таким же именем и структурой.

In [2]:
url = (
    "https://raw.githubusercontent.com/pandas-dev"
    "/pandas/main/pandas/tests/io/data/csv/tips.csv"
)

tips = pd.read_csv(url)

tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


## Copies vs. in place operations

Большинство операций pandas возвращают копии `Series`/`DataFrame`. Чтобы изменения «закрепились», вам нужно либо назначить новую переменную
```python
sorted_df = df.sort_values("col1")
```
или переписать исходную переменную:
```python
df = df.sort_values("col1")
```

Вы увидите аргумент `inplace=True` или `copy=False`, доступный для некоторых методов:
```python
df.replace(5, inplace=Истина)
```
Идет активное обсуждение отказа от поддержки и удаления на месте и копирования для большинства методов (например, dropna), за исключением очень небольшого подмножества методов (включая replace). Оба ключевых слова больше не понадобятся в контексте копирования при записи. Предложение можно найти [здесь](https://github.com/pandas-dev/pandas/pull/51466).

## SELECT

В SQL выбор осуществляется с помощью списка столбцов, разделенных запятыми, которые вы хотите выбрать (или *, чтобы выбрать все столбцы):
```SQL
SELECT total_bill, tip, smoker, time
FROM tips;
```

С пандами выбор столбца осуществляется путем передачи списка имен столбцов в ваш `DataFrame`:

In [3]:
tips[["total_bill", "tip", "smoker", "time"]]

,total_bill,tip,smoker,time
0,16.99,1.01,No,Dinner
1,10.34,1.66,No,Dinner
2,21.01,3.50,No,Dinner
3,23.68,3.31,No,Dinner
4,24.59,3.61,No,Dinner
...,...,...,...,...
239,29.03,5.92,No,Dinner
240,27.18,2.00,Yes,Dinner
241,22.67,2.00,Yes,Dinner
242,17.82,1.75,No,Dinner


Вызов `DataFrame` без списка имен столбцов отобразит все столбцы (аналогично SQL `*`).

В SQL вы можете добавить вычисляемый столбец:
```SQL
SELECT *, tip/total_bill as tip_rate
FROM tips;
```
С pandas вы можете использовать метод `DataFrame.assign()` для добавления нового столбца:

In [4]:
tips.assign(tip_rate=tips["tip"] / tips["total_bill"])

,total_bill,tip,sex,smoker,day,time,size,tip_rate
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204


## WHERE

Фильтрация в SQL выполняется с помощью предложения `WHERE`.
```SQL
SELECT *
FROM tips
WHERE time = 'Dinner';
```
`DataFrame` можно фильтровать несколькими способами; наиболее интуитивным из которых является использование булевой индексации.

In [5]:
tips[tips["total_bill"] > 10]

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


Приведенный выше оператор просто передает `Series` объектов `True`/`False` в `DataFrame`, возвращая все строки с `True`.

In [6]:
is_dinner = tips["time"] == "Dinner"

is_dinner

0      True
1      True
2      True
3      True
4      True
       ... 
239    True
240    True
241    True
242    True
243    True
Name: time, Length: 244, dtype: bool

In [7]:
is_dinner.value_counts()

True     176
False     68
Name: time, dtype: int64

In [8]:
tips[is_dinner]

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


Так же, как операторы `OR` и `AND` в SQL, в `DataFrame` можно передать несколько условий, используя `|` (ИЛИ) и `&` (И).

Чаевые на сумму более 5 долларов за ужин:
```SQL
SELECT *
FROM tips
WHERE time = 'Dinner' AND tip > 5.00;
```

In [9]:
tips[(tips["time"] == "Dinner") & (tips["tip"] > 5.00)]

,total_bill,tip,sex,smoker,day,time,size
23,39.42,7.58,Male,No,Sat,Dinner,4
44,30.40,5.60,Male,No,Sun,Dinner,4
47,32.40,6.00,Male,No,Sun,Dinner,4
52,34.81,5.20,Female,No,Sun,Dinner,4
59,48.27,6.73,Male,No,Sat,Dinner,4
116,29.93,5.07,Male,No,Sun,Dinner,4
155,29.85,5.14,Female,No,Sun,Dinner,5
170,50.81,10.00,Male,Yes,Sat,Dinner,3
172,7.25,5.15,Male,Yes,Sun,Dinner,2
181,23.33,5.65,Male,Yes,Sun,Dinner,2


Чаевые от вечеринок, состоящих не менее чем из 5 посетителей ИЛИ общая сумма счета превышает 45 долларов США:
```SQL
SELECT *
FROM tips
WHERE size >= 5 OR total_bill > 45;
```

In [10]:
tips[(tips["size"] >= 5) | (tips["total_bill"] > 45)]

,total_bill,tip,sex,smoker,day,time,size
59,48.27,6.73,Male,No,Sat,Dinner,4
125,29.80,4.20,Female,No,Thur,Lunch,6
141,34.30,6.70,Male,No,Thur,Lunch,6
142,41.19,5.00,Male,No,Thur,Lunch,5
143,27.05,5.00,Female,No,Thur,Lunch,6
155,29.85,5.14,Female,No,Sun,Dinner,5
156,48.17,5.00,Male,No,Sun,Dinner,6
170,50.81,10.00,Male,Yes,Sat,Dinner,3
182,45.35,3.50,Male,Yes,Sun,Dinner,3
185,20.69,5.00,Male,No,Sun,Dinner,5


Проверка NULL выполняется с помощью методов `notna()` и `isna()`.

In [11]:
frame = pd.DataFrame(
    {"col1": ["A", "B", np.NaN, "C", "D"], "col2": ["F", np.NaN, "G", "H", "I"]}
)


frame

,col1,col2
0,A,F
1,B,NaN
2,NaN,G
3,C,H
4,D,I


Предположим, у нас есть таблица той же структуры, что и наш `DataFrame` выше. Мы можем видеть только записи, где `col2 IS NULL` со следующим запросом:
```SQL
SELECT *
FROM frame
WHERE col2 IS NULL;
```

In [12]:
frame[frame["col2"].isna()]

,col1,col2
1,B,NaN


Получение элементов, где `col1 IS NOT NULL`, может быть выполнено с помощью `notna()`.
```SQL
SELECT *
FROM frame
WHERE col1 IS NOT NULL;
```

In [13]:
frame[frame["col1"].notna()]

,col1,col2
0,A,F
1,B,NaN
3,C,H
4,D,I


# GROUP BY

В pandas операции SQL `GROUP BY` выполняются с использованием метода `groupby()`. `groupby(`) обычно относится к процессу, в котором мы хотели бы разделить набор данных на группы, применить некоторую функцию (обычно агрегацию), а затем объединить группы вместе.

Обычной операцией SQL будет получение количества записей в каждой группе по всему набору данных. Например, запрос, дающий нам количество чаевых, разделенных по половому признаку:
```SQL
SELECT sex, count(*)
FROM tips
GROUP BY sex;
/*
Female     87
Male      157
*/
```

In [14]:
tips.groupby("sex").size()

sex
Female     87
Male      157
dtype: int64

Обратите внимание, что в коде pandas мы использовали `size()`, а не `count()`. Это связано с тем, что `count()` применяет функцию к *каждому* столбцу, возвращая количество записей, которые `NOT NULL`.

In [15]:
tips.groupby("sex").count()

,total_bill,tip,smoker,day,time,size
sex,,,,,,
Female,87,87,87,87,87,87
Male,157,157,157,157,157,157


В качестве альтернативы мы могли бы применить метод `count()` к отдельному столбцу:

In [16]:
tips.groupby("sex")["total_bill"].count()

sex
Female     87
Male      157
Name: total_bill, dtype: int64

Несколько функций также могут быть применены одновременно. Например, предположим, что мы хотели бы увидеть, как сумма чаевых отличается в зависимости от дня недели — `agg()` позволяет вам передать словарь в ваш сгруппированный DataFrame, указывая, какие функции следует применять к определенным столбцам.
```SQL
SELECT day, AVG(tip), COUNT(*)
FROM tips
GROUP BY day;
/*
Fri   2.734737   19
Sat   2.993103   87
Sun   3.255132   76
Thu  2.771452   62
*/
```

In [17]:
tips.groupby("day").agg({"tip": np.mean, "day": np.size})

,tip,day
day,,
Fri,2.734737,19
Sat,2.993103,87
Sun,3.255132,76
Thur,2.771452,62


Группировка по нескольким столбцам выполняется путем передачи списка столбцов в метод `groupby()`.
```SQL
SELECT smoker, day, COUNT(*), AVG(tip)
FROM tips
GROUP BY smoker, day;
/*
smoker day
No     Fri      4  2.812500
       Sat     45  3.102889
       Sun     57  3.167895
       Thu    45  2.673778
Yes    Fri     15  2.714000
       Sat     42  2.875476
       Sun     19  3.516842
       Thu    17  3.030000
*/
```

In [18]:
tips.groupby(["smoker", "day"]).agg({"tip": [np.size, np.mean]})

tip          
            size      mean
smoker day                
No     Fri     4  2.812500
       Sat    45  3.102889
       Sun    57  3.167895
       Thur   45  2.673778
Yes    Fri    15  2.714000
       Sat    42  2.875476
       Sun    19  3.516842
       Thur   17  3.030000

## JOIN

СОЕДИНЕНИЯ могут выполняться с помощью `join()` или `merge()`. По умолчанию `join(`) будет присоединяться к DataFrames по их индексам. Каждый метод имеет параметры, позволяющие указать тип выполняемого соединения (LEFT, RIGHT, INNER, FULL) или столбцы для объединения (имена столбцов или индексы).

Если оба ключевых столбца содержат строки, в которых ключ является нулевым значением, эти строки будут сопоставлены друг с другом. Это отличается от обычного поведения соединения SQL и может привести к неожиданным результатам.

In [19]:
df1 = pd.DataFrame({"key": ["A", "B", "C", "D"], "value": np.random.randn(4)})

df2 = pd.DataFrame({"key": ["B", "D", "D", "E"], "value": np.random.randn(4)})

Теперь давайте рассмотрим различные типы JOIN'ов.

### INNER JOIN
```SQL
SELECT *
FROM df1
INNER JOIN df2
  ON df1.key = df2.key;
```

In [20]:
# merge performs an INNER JOIN by default
pd.merge(df1, df2, on="key")

,key,value_x,value_y
0,B,0.697381,0.282075
1,D,-1.249893,-1.063754
2,D,-1.249893,0.580715


`merge()` также предлагает параметры для случаев, когда вы хотите соединить один столбец DataFrame с индексом другого DataFrame.

### LEFT OUTER JOIN
Показать все из `df1`.
```SQL
SELECT *
FROM df1
LEFT OUTER JOIN df2
  ON df1.key = df2.key;
```

In [21]:
pd.merge(df1, df2, on="key", how="left")

,key,value_x,value_y
0,A,0.642109,NaN
1,B,0.697381,0.282075
2,C,-0.066624,NaN
3,D,-1.249893,-1.063754
4,D,-1.249893,0.580715


### RIGHT JOIN
Показать все из `df2`.
```SQL
SELECT *
FROM df1
RIGHT OUTER JOIN df2
  ON df1.key = df2.key;
```

In [22]:
pd.merge(df1, df2, on="key", how="right")

,key,value_x,value_y
0,B,0.697381,0.282075
1,D,-1.249893,-1.063754
2,D,-1.249893,0.580715
3,E,NaN,0.181319


### FULL JOIN

pandas также допускает ПОЛНЫЕ СОЕДИНЕНИЯ, которые отображают обе стороны набора данных, независимо от того, находят ли совпадения соединенные столбцы. На момент написания ПОЛНЫЕ СОЕДИНЕНИЯ не поддерживаются во всех СУБД (MySQL).

Показать все записи из обеих таблиц.
```SQL
SELECT *
FROM df1
FULL OUTER JOIN df2
  ON df1.key = df2.key;
```

In [23]:
pd.merge(df1, df2, on="key", how="outer")

,key,value_x,value_y
0,A,0.642109,NaN
1,B,0.697381,0.282075
2,C,-0.066624,NaN
3,D,-1.249893,-1.063754
4,D,-1.249893,0.580715
5,E,NaN,0.181319


## UNION
`UNION ALL` делается через `concat()`.

In [24]:
df1 = pd.DataFrame(
    {"city": ["Chicago", "San Francisco", "New York City"], "rank": range(1, 4)}
)


df2 = pd.DataFrame(
    {"city": ["Chicago", "Boston", "Los Angeles"], "rank": [1, 4, 5]}
)

```SQL
SELECT city, rank
FROM df1
UNION ALL
SELECT city, rank
FROM df2;
/*
         city  rank
      Chicago     1
San Francisco     2
New York City     3
      Chicago     1
       Boston     4
  Los Angeles     5
*/
```

In [25]:
pd.concat([df1, df2])

,city,rank
0,Chicago,1
1,San Francisco,2
2,New York City,3
0,Chicago,1
1,Boston,4
2,Los Angeles,5


`UNION` в SQL похож на `UNION ALL`, однако `UNION` удалит повторяющиеся строки.
```SQL
SELECT city, rank
FROM df1
UNION
SELECT city, rank
FROM df2;
-- notice that there is only one Chicago record this time
/*
         city  rank
      Chicago     1
San Francisco     2
New York City     3
       Boston     4
  Los Angeles     5
*/
```
В pandas вы можете использовать `concat()` в сочетании с `drop_duplicates()`.

In [26]:
pd.concat([df1, df2]).drop_duplicates()

,city,rank
0,Chicago,1
1,San Francisco,2
2,New York City,3
1,Boston,4
2,Los Angeles,5


## LIMIT
```SQL
SELECT * FROM tips
LIMIT 10;
```

In [27]:
tips.head(10)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
5,25.29,4.71,Male,No,Sun,Dinner,4
6,8.77,2.00,Male,No,Sun,Dinner,2
7,26.88,3.12,Male,No,Sun,Dinner,4
8,15.04,1.96,Male,No,Sun,Dinner,2
9,14.78,3.23,Male,No,Sun,Dinner,2


## Эквиваленты pandas для некоторых аналитических и агрегатных функций SQL

### Верхние n строк со смещением
```SQL
-- MySQL
SELECT * FROM tips
ORDER BY tip DESC
LIMIT 10 OFFSET 5;
```

In [28]:
tips.nlargest(10 + 5, columns="tip").tail(10)

,total_bill,tip,sex,smoker,day,time,size
183,23.17,6.50,Male,Yes,Sun,Dinner,4
214,28.17,6.50,Female,Yes,Sat,Dinner,3
47,32.40,6.00,Male,No,Sun,Dinner,4
239,29.03,5.92,Male,No,Sat,Dinner,3
88,24.71,5.85,Male,No,Thur,Lunch,2
181,23.33,5.65,Male,Yes,Sun,Dinner,2
44,30.40,5.60,Male,No,Sun,Dinner,4
52,34.81,5.20,Female,No,Sun,Dinner,4
85,34.83,5.17,Female,No,Thur,Lunch,4
211,25.89,5.16,Male,Yes,Sat,Dinner,4


### Верхние n строк в группе
```SQL
-- Oracle's ROW_NUMBER() analytic function
SELECT * FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER(PARTITION BY day ORDER BY total_bill DESC) AS rn
  FROM tips t
)
WHERE rn < 3
ORDER BY day, rn;
```

In [29]:
(
    tips.assign(
        rn=tips.sort_values(["total_bill"], ascending=False)
        .groupby(["day"])
        .cumcount()
        + 1
    )
    .query("rn < 3")
    .sort_values(["day", "rn"])
)

,total_bill,tip,sex,smoker,day,time,size,rn
95,40.17,4.73,Male,Yes,Fri,Dinner,4,1
90,28.97,3.00,Male,Yes,Fri,Dinner,2,2
170,50.81,10.00,Male,Yes,Sat,Dinner,3,1
212,48.33,9.00,Male,No,Sat,Dinner,4,2
156,48.17,5.00,Male,No,Sun,Dinner,6,1
182,45.35,3.50,Male,Yes,Sun,Dinner,3,2
197,43.11,5.00,Female,Yes,Thur,Lunch,4,1
142,41.19,5.00,Male,No,Thur,Lunch,5,2


То же самое с использованием функции `rank(method='first')`

In [30]:
(
    tips.assign(
        rnk=tips.groupby(["day"])["total_bill"].rank(
            method="first", ascending=False
        )
    )
    .query("rnk < 3")
    .sort_values(["day", "rnk"])
)

,total_bill,tip,sex,smoker,day,time,size,rnk
95,40.17,4.73,Male,Yes,Fri,Dinner,4,1.0
90,28.97,3.00,Male,Yes,Fri,Dinner,2,2.0
170,50.81,10.00,Male,Yes,Sat,Dinner,3,1.0
212,48.33,9.00,Male,No,Sat,Dinner,4,2.0
156,48.17,5.00,Male,No,Sun,Dinner,6,1.0
182,45.35,3.50,Male,Yes,Sun,Dinner,3,2.0
197,43.11,5.00,Female,Yes,Thur,Lunch,4,1.0
142,41.19,5.00,Male,No,Thur,Lunch,5,2.0


```SQL
-- Oracle's RANK() analytic function
SELECT * FROM (
  SELECT
    t.*,
    RANK() OVER(PARTITION BY sex ORDER BY tip) AS rnk
  FROM tips t
  WHERE tip < 2
)
WHERE rnk < 3
ORDER BY sex, rnk;
```
Давайте найдем чаевые (rank < 3) по половому признаку для (tips < 2). Отметим, что когда используется `rank(method='min')` функция, `rnk_min` остается тем же самым для тех же чаевых(как RANK() function)

In [31]:
(
    tips[tips["tip"] < 2]
    .assign(rnk_min=tips.groupby(["sex"])["tip"].rank(method="min"))
    .query("rnk_min < 3")
    .sort_values(["sex", "rnk_min"])
)

,total_bill,tip,sex,smoker,day,time,size,rnk_min
67,3.07,1.00,Female,Yes,Sat,Dinner,1,1.0
92,5.75,1.00,Female,Yes,Fri,Dinner,2,1.0
111,7.25,1.00,Female,No,Sat,Dinner,1,1.0
236,12.60,1.00,Male,Yes,Sat,Dinner,2,1.0
237,32.83,1.17,Male,Yes,Sat,Dinner,2,2.0


## UPDATE
```SQL
UPDATE tips
SET tip = tip*2
WHERE tip < 2;
```

In [32]:
tips.loc[tips["tip"] < 2, "tip"] *= 2

## DELETE
```SQL
DELETE FROM tips
WHERE tip > 9;
```
В pandas мы выбираем строки, которые должны остаться, а не удаляем их:

In [33]:
tips = tips.loc[tips["tip"] <= 9]